<a href="https://colab.research.google.com/github/itsmeneeraj/alpha/blob/main/dynamicprog2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd


# Define constants and parameters
T = 10  # Time horizon in years
G = 200000  # Target wealth
W0 = 100000  # Initial wealth
monthly_cash_flow = 1000 # Monthly Cash Flow
n_assets = 2  # Number of assets (Equity & Debt)
mu = np.array([0.07, 0.12])  # Expected returns (Nifty500 and Bond Yield)
sigma = np.array([0.1, 0.2])  # Volatilities (Nifty500 and Bond Yield)
corr_matrix = np.array([[1, 0.3], [0.2,0.5]])  # Correlation matrix
cov_matrix = np.outer(sigma, sigma) * corr_matrix  # Covariance matrix

# Time and wealth discretization
months = T*12 # Total Number of months
time_steps = np.arange(0, months+1)
wealth_grid = np.linspace(0, 2*G, 100)

# Initialize dynamic programming table
dp_table = np.zeros((len(time_steps), len(wealth_grid)))
dp_table[-1, :] = (wealth_grid >= G).astype(int)  # Final step

# Backward recursion
for t in reversed(range(T)):
    for i, W in enumerate(wealth_grid):
        max_prob = 0
        for a in np.linspace(0, 1, 11):  # Portfolio allocations
            # Portfolio mean and variance
            portfolio_mu = a * mu[0] + (1-a) * mu[1]
            portfolio_sigma = np.sqrt(a**2 * sigma[0]**2 + (1-a)**2 * sigma[1]**2 + 2*a*(1-a)*cov_matrix[0,1])

            # Expected wealth at next time step
            expected_wealth = W * np.exp((portfolio_mu - 0.5 * portfolio_sigma**2) + portfolio_sigma * np.random.normal(0, 1))
            j = np.searchsorted(wealth_grid, expected_wealth, side='right') - 1
            j = np.clip(j, 0, len(wealth_grid)-1)

            # Probability of reaching goal
            prob = dp_table[t+1, j]
            max_prob = max(max_prob, prob)

        dp_table[t, i] = max_prob

# Optimal strategy
optimal_strategy = []
for t in range(T):
    for i, W in enumerate(wealth_grid):
        max_prob = 0
        best_allocation = None
        for a in np.linspace(0, 1, 11):
            portfolio_mu = a * mu[0] + (1-a) * mu[1]
            portfolio_sigma = np.sqrt(a**2 * sigma[0]**2 + (1-a)**2 * sigma[1]**2 + 2*a*(1-a)*cov_matrix[0,1])
            expected_wealth = W * np.exp((portfolio_mu - 0.5 * portfolio_sigma**2) + portfolio_sigma * np.random.normal(0, 1))
            j = np.searchsorted(wealth_grid, expected_wealth, side='right') - 1
            j = np.clip(j, 0, len(wealth_grid)-1)
            prob = dp_table[t+1, j]
            if prob > max_prob:
                max_prob = prob
                best_allocation = a
        optimal_strategy.append((t, W, best_allocation))

print("Optimal Strategy:", optimal_strategy)




Optimal Strategy: [(0, 0.0, None), (0, 4040.40404040404, None), (0, 8080.80808080808, None), (0, 12121.21212121212, None), (0, 16161.61616161616, None), (0, 20202.0202020202, None), (0, 24242.42424242424, None), (0, 28282.828282828283, None), (0, 32323.23232323232, None), (0, 36363.63636363636, None), (0, 40404.0404040404, None), (0, 44444.444444444445, None), (0, 48484.84848484848, None), (0, 52525.25252525252, None), (0, 56565.656565656565, None), (0, 60606.0606060606, None), (0, 64646.46464646464, None), (0, 68686.86868686868, None), (0, 72727.27272727272, None), (0, 76767.67676767676, None), (0, 80808.0808080808, None), (0, 84848.48484848485, None), (0, 88888.88888888889, None), (0, 92929.29292929292, None), (0, 96969.69696969696, None), (0, 101010.101010101, None), (0, 105050.50505050505, None), (0, 109090.90909090909, None), (0, 113131.31313131313, None), (0, 117171.71717171716, None), (0, 121212.1212121212, None), (0, 125252.52525252524, None), (0, 129292.92929292929, None), (0,